### Here we compare the different validation techniques on an imagenet model already trained to 93%

In [1]:
import sys, os, shutil, time, warnings
from datetime import datetime
from pathlib import Path
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
from torch.utils.data.sampler import Sampler
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from tqdm import tqdm
import urllib.request
import pandas as pd

In [15]:
cudnn.benchmark = True
data = Path.home()/'data/imagenet'
workers = 7
valdir = os.path.join(data, 'validation')
batch_size = 64
fp16 = False

## Create Image to Aspect ratio mapping

In [4]:
# Step 1: sort images by aspect ratio
def sort_ar(valdir):
    idx2ar_file = data/'sorted_idxar.p'
    if os.path.isfile(idx2ar_file): return pickle.load(open(idx2ar_file, 'rb'))
    print('Creating AR indexes. Please be patient this may take a couple minutes...')
    val_dataset = datasets.ImageFolder(valdir)
    sizes = [img[0].size for img in tqdm(val_dataset, total=len(val_dataset))]
    idx_ar = [(i, round(s[0]/s[1], 5)) for i,s in enumerate(sizes)]
    sorted_idxar = sorted(idx_ar, key=lambda x: x[1])
    pickle.dump(sorted_idxar, open(idx2ar_file, 'wb'))
    return sorted_idxar

# Step 2: chunk images by batch size. This way we can crop each image to the batch aspect ratio mean 
def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

# Step 3: map image index to batch aspect ratio mean so our transform function knows where to crop
def map_idx2ar(idx_ar_sorted, batch_size):
    ar_chunks = list(chunks(idx_ar_sorted, batch_size))
    idx2ar = {}
    ar_means = []
    for chunk in ar_chunks:
        idxs, ars = list(zip(*chunk))
        mean = round(np.mean(ars), 5)
        ar_means.append(mean)
        for idx in idxs:
            idx2ar[idx] = mean
    return idx2ar, ar_means

In [5]:
idx_ar_sorted = sort_ar(valdir)

#### OR just download it

In [6]:
idx2ar_path = data/'sorted_idxar.p'
url = 'https://s3-us-west-2.amazonaws.com/ashaw-fastai-imagenet/sorted_idxar.p'
if not os.path.exists(idx2ar_path): urllib.request.urlretrieve(url, idx2ar_path)
idx_ar_sorted = sort_ar(valdir)

## Dataset Preparation

In [7]:
class ValDataset(datasets.ImageFolder):
    def __init__(self, root, transform=None, target_transform=None):
        super().__init__(root, transform, target_transform)
    def __getitem__(self, index):
        path, target = self.imgs[index]
        sample = self.loader(path)
        if self.transform is not None:
            for tfm in self.transform:
                if isinstance(tfm, RectangularCropTfm): sample = tfm(sample, index)
                else: sample = tfm(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

    
# Essentially a sequential sampler
class SequentialIndexSampler(Sampler):
    def __init__(self, indices): self.indices = indices
    def __len__(self): return len(self.indices)
    def __iter__(self): return iter(self.indices)
    


In [8]:
class RectangularCropTfm(object):
    def __init__(self, idx2ar, target_size):
        self.idx2ar, self.target_size = idx2ar, target_size
    def __call__(self, img, idx):
        target_ar = self.idx2ar[idx]
        if target_ar < 1: 
            w = int(self.target_size/target_ar)
            size = (w//8*8, self.target_size)
        else: 
            h = int(self.target_size*target_ar)
            size = (self.target_size, h//8*8)
        return transforms.functional.center_crop(img, size)

## Validation Function with TTA

In [9]:
def validate(val_loader, model, criterion, aug_loader=None, num_augmentations=0):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    start_time = datetime.now()

    model.eval()
    end = time.time()

    val_iter = iter(val_loader)
    aug_iters = [iter(aug_loader) for i in range(num_augmentations)]
    prec5_arr = []
    for i in range(len(val_loader)):
        def get_output(dl_iter):
            input,target = next(dl_iter)
            input, target = input.cuda(), target.cuda()
            if fp16: input = input.half()

            # compute output
            with torch.no_grad():
                output = model(Variable(input))
                loss = criterion(output, Variable(target))
            return output, loss, input, target
        
        # Normal Validation
        output,loss,input,target = get_output(val_iter)
        
        # TTA
        for aug_iter in aug_iters:
            o,l,_,_ = get_output(aug_iter)
            output.add_(o)
            loss.add_(l)
        loss.div_(num_augmentations+1)
        
        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        reduced_loss = loss.data
            
        losses.update(to_python_float(reduced_loss), input.size(0))
        top1.update(to_python_float(prec1), input.size(0))
        top5.update(to_python_float(prec5), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if ((i+1)%50 == 0) or ((i+1) == len(val_loader)):
            prec5_arr.append(top5.val)
            output = ('Test: [{0}/{1}]\t' \
                    + 'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t' \
                    + 'Loss {loss.val:.4f} ({loss.avg:.4f})\t' \
                    + 'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t' \
                    + 'Prec@5 {top5.val:.3f} ({top5.avg:.3f})').format(
                    i+1, len(val_loader), batch_time=batch_time, loss=losses,
                    top1=top1, top5=top5)
            print(output)

    time_diff = datetime.now()-start_time
    print(f'Total Time:{float(time_diff.total_seconds() / 3600.0)}\t Top 5 Accuracy: {top5.avg:.3f}\n')
    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))

    return prec5_arr

# item() is a recent addition, so this helps with backward compatibility.
def to_python_float(t):
    if hasattr(t, 'item'): return t.item()
    else: return t[0]
    
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self): self.reset()
    def reset(self): self.val = self.avg = self.sum = self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

### Get pretrained resnet model

In [16]:
import resnet
model = resnet.resnet50(pretrained=True)
model = model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
if fp16:
    from fp16util import network_to_half
    model = network_to_half(model)

### Create Dataset that supports Rectangles

In [12]:
class ValDataset(datasets.ImageFolder):
    def __init__(self, root, transform=None, target_transform=None):
        super().__init__(root, transform, target_transform)
    def __getitem__(self, index):
        path, target = self.imgs[index]
        sample = self.loader(path)
        if self.transform is not None:
            for tfm in self.transform:
                if isinstance(tfm, RectangularCropTfm): sample = tfm(sample, index)
                else: sample = tfm(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target


### Global dataset settings

In [13]:
val_bs = 64
target_size = 288

idx_sorted, _ = zip(*idx_ar_sorted)
idx2ar, ar_means = map_idx2ar(idx_ar_sorted, val_bs)
val_sampler_ar = SequentialIndexSampler(idx_sorted)

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
tensor_tfm = [transforms.ToTensor(), normalize]

### Test Original Validation Technique

This was the validation technique used in fast.ai's original [DAWNBenchmark](https://dawn.cs.stanford.edu/benchmark/ImageNet/train.html)  
Resize Image 1.14x -> Crop to target size (288)

In [18]:
val_tfms = [transforms.Resize(int(target_size*1.14)), transforms.CenterCrop(target_size)] + tensor_tfm
val_dataset = datasets.ImageFolder(valdir,  transforms.Compose(val_tfms))

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

orig_prec5 = validate(val_loader, model, criterion)

Test: [50/391]	Time 0.107 (0.234)	Loss 1.2314 (1.0281)	Prec@1 72.656 (74.433)	Prec@5 90.625 (92.019)
Test: [100/391]	Time 0.106 (0.215)	Loss 0.8887 (0.9856)	Prec@1 75.781 (75.433)	Prec@5 93.750 (92.605)
Test: [150/391]	Time 0.107 (0.211)	Loss 0.6523 (0.9521)	Prec@1 83.594 (76.247)	Prec@5 96.094 (93.031)
Test: [200/391]	Time 0.124 (0.209)	Loss 0.7725 (0.9011)	Prec@1 82.031 (77.484)	Prec@5 96.094 (93.528)
Test: [250/391]	Time 0.229 (0.207)	Loss 0.7354 (0.9047)	Prec@1 81.250 (77.344)	Prec@5 94.531 (93.657)
Test: [300/391]	Time 0.120 (0.206)	Loss 0.8486 (0.9344)	Prec@1 79.688 (76.806)	Prec@5 96.875 (93.218)
Test: [350/391]	Time 0.443 (0.207)	Loss 0.9238 (0.9248)	Prec@1 77.344 (76.945)	Prec@5 89.062 (93.354)
~~0.022758550833333335	93.430

 * Prec@1 76.914 Prec@5 93.430


### Test Fast.Ai Rectangular Validation

## Batch size 128 

In [22]:
val_ar_tfms = [transforms.Resize(int(target_size*1.14)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_rs_prec5 = validate(val_loader, model, criterion)

Test: [50/391]	Time 0.138 (0.681)	Loss 1.0703 (1.0479)	Prec@1 70.312 (76.109)	Prec@5 91.406 (93.328)
Test: [100/391]	Time 0.126 (0.626)	Loss 0.7603 (1.0027)	Prec@1 75.781 (76.578)	Prec@5 98.438 (93.477)
Test: [150/391]	Time 1.666 (0.558)	Loss 0.9561 (0.9680)	Prec@1 75.000 (76.932)	Prec@5 92.969 (93.656)
Test: [200/391]	Time 0.509 (0.489)	Loss 0.8340 (0.9238)	Prec@1 82.031 (77.902)	Prec@5 96.094 (94.039)
Test: [250/391]	Time 0.142 (0.440)	Loss 0.8906 (0.9364)	Prec@1 78.906 (77.703)	Prec@5 93.750 (94.025)
Test: [300/391]	Time 0.149 (0.403)	Loss 1.0117 (0.9608)	Prec@1 79.688 (77.180)	Prec@5 92.188 (93.667)
Test: [350/391]	Time 0.162 (0.404)	Loss 0.7207 (0.9545)	Prec@1 82.812 (77.299)	Prec@5 96.875 (93.821)
Test: [391/391]	Time 3.615 (0.449)	Loss 1.0947 (0.9550)	Prec@1 76.250 (77.352)	Prec@5 90.000 (93.886)
Total Time:0.04871557305555555	 Top 5 Accuracy: 93.886

 * Prec@1 77.352 Prec@5 93.886


## Batch size 64 

### FP16 (BN is also fp16)

In [25]:
val_ar_tfms = [transforms.Resize(int(target_size*1.14)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=64, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_rs_prec5 = validate(val_loader, model, criterion)

Test: [50/782]	Time 0.074 (0.513)	Loss 1.2061 (1.0429)	Prec@1 67.188 (76.031)	Prec@5 90.625 (94.000)
Test: [100/782]	Time 0.125 (0.327)	Loss 1.4004 (1.0478)	Prec@1 59.375 (76.172)	Prec@5 87.500 (93.422)
Test: [150/782]	Time 0.069 (0.267)	Loss 0.7969 (1.0624)	Prec@1 81.250 (75.604)	Prec@5 95.312 (92.948)
Test: [200/782]	Time 0.058 (0.267)	Loss 0.5542 (1.0024)	Prec@1 85.938 (76.594)	Prec@5 100.000 (93.539)
Test: [250/782]	Time 0.067 (0.240)	Loss 0.7871 (0.9978)	Prec@1 81.250 (76.412)	Prec@5 93.750 (93.481)
Test: [300/782]	Time 0.075 (0.237)	Loss 0.9624 (0.9680)	Prec@1 76.562 (76.948)	Prec@5 92.188 (93.698)
Test: [350/782]	Time 0.071 (0.224)	Loss 0.8110 (0.9422)	Prec@1 85.938 (77.589)	Prec@5 95.312 (93.902)
Test: [400/782]	Time 0.071 (0.208)	Loss 0.9170 (0.9237)	Prec@1 79.688 (77.914)	Prec@5 93.750 (94.078)
Test: [450/782]	Time 0.286 (0.198)	Loss 1.1611 (0.9234)	Prec@1 64.062 (78.049)	Prec@5 92.188 (94.243)
Test: [500/782]	Time 0.075 (0.189)	Loss 0.3828 (0.9363)	Prec@1 90.625 (77.713)	Pre

### Network to half

In [14]:
val_ar_tfms = [transforms.Resize(int(target_size*1.14)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=64, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_rs_prec5 = validate(val_loader, model, criterion)

Test: [50/782]	Time 0.069 (0.523)	Loss 1.2061 (1.0429)	Prec@1 67.188 (76.062)	Prec@5 90.625 (94.000)
Test: [100/782]	Time 0.209 (0.335)	Loss 1.3994 (1.0478)	Prec@1 59.375 (76.172)	Prec@5 87.500 (93.406)
Test: [150/782]	Time 0.064 (0.270)	Loss 0.7974 (1.0625)	Prec@1 81.250 (75.604)	Prec@5 95.312 (92.938)
Test: [200/782]	Time 0.069 (0.268)	Loss 0.5547 (1.0025)	Prec@1 85.938 (76.594)	Prec@5 100.000 (93.531)
Test: [250/782]	Time 0.062 (0.243)	Loss 0.7871 (0.9978)	Prec@1 81.250 (76.412)	Prec@5 93.750 (93.469)
Test: [300/782]	Time 0.072 (0.238)	Loss 0.9624 (0.9680)	Prec@1 76.562 (76.953)	Prec@5 92.188 (93.688)
Test: [350/782]	Time 0.065 (0.225)	Loss 0.8110 (0.9422)	Prec@1 85.938 (77.594)	Prec@5 95.312 (93.893)
Test: [400/782]	Time 0.071 (0.210)	Loss 0.9170 (0.9237)	Prec@1 79.688 (77.918)	Prec@5 93.750 (94.070)
Test: [450/782]	Time 0.343 (0.199)	Loss 1.1611 (0.9234)	Prec@1 64.062 (78.049)	Prec@5 92.188 (94.236)
Test: [500/782]	Time 0.065 (0.190)	Loss 0.3823 (0.9363)	Prec@1 90.625 (77.709)	Pre

## Full precision

In [17]:
val_ar_tfms = [transforms.Resize(int(target_size*1.14)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=64, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_rs_prec5 = validate(val_loader, model, criterion)

Test: [50/782]	Time 0.136 (0.565)	Loss 1.2061 (1.0429)	Prec@1 65.625 (75.938)	Prec@5 90.625 (94.031)
Test: [100/782]	Time 0.129 (0.372)	Loss 1.3998 (1.0477)	Prec@1 59.375 (76.156)	Prec@5 87.500 (93.422)
Test: [150/782]	Time 0.126 (0.305)	Loss 0.7977 (1.0624)	Prec@1 81.250 (75.594)	Prec@5 95.312 (92.958)
Test: [200/782]	Time 0.106 (0.305)	Loss 0.5542 (1.0023)	Prec@1 85.938 (76.617)	Prec@5 100.000 (93.547)
Test: [250/782]	Time 0.111 (0.275)	Loss 0.7883 (0.9978)	Prec@1 81.250 (76.431)	Prec@5 93.750 (93.487)
Test: [300/782]	Time 0.134 (0.272)	Loss 0.9626 (0.9680)	Prec@1 76.562 (76.969)	Prec@5 92.188 (93.703)
Test: [350/782]	Time 0.129 (0.262)	Loss 0.8109 (0.9421)	Prec@1 85.938 (77.612)	Prec@5 95.312 (93.902)
Test: [400/782]	Time 0.130 (0.245)	Loss 0.9166 (0.9237)	Prec@1 79.688 (77.934)	Prec@5 93.750 (94.074)
Test: [450/782]	Time 0.129 (0.233)	Loss 1.1614 (0.9234)	Prec@1 64.062 (78.066)	Prec@5 92.188 (94.247)
Test: [500/782]	Time 0.130 (0.222)	Loss 0.3823 (0.9363)	Prec@1 90.625 (77.728)	Pre

### Full Precision -  AR 1.2

In [18]:
val_ar_tfms = [transforms.Resize(int(target_size*1.2)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=64, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_rs_prec5 = validate(val_loader, model, criterion)

Test: [50/782]	Time 0.136 (0.181)	Loss 1.2006 (1.0500)	Prec@1 67.188 (75.500)	Prec@5 87.500 (93.844)
Test: [100/782]	Time 0.129 (0.162)	Loss 1.4244 (1.0571)	Prec@1 57.812 (75.750)	Prec@5 87.500 (93.250)
Test: [150/782]	Time 0.126 (0.155)	Loss 0.8122 (1.0692)	Prec@1 76.562 (75.229)	Prec@5 95.312 (92.740)
Test: [200/782]	Time 0.112 (0.146)	Loss 0.5240 (1.0075)	Prec@1 87.500 (76.344)	Prec@5 100.000 (93.305)
Test: [250/782]	Time 0.107 (0.137)	Loss 0.8130 (1.0045)	Prec@1 81.250 (76.150)	Prec@5 95.312 (93.213)
Test: [300/782]	Time 0.121 (0.135)	Loss 0.9275 (0.9737)	Prec@1 75.000 (76.620)	Prec@5 90.625 (93.438)
Test: [350/782]	Time 0.129 (0.135)	Loss 0.8336 (0.9480)	Prec@1 84.375 (77.263)	Prec@5 92.188 (93.634)
Test: [400/782]	Time 0.129 (0.137)	Loss 0.9164 (0.9290)	Prec@1 81.250 (77.715)	Prec@5 95.312 (93.859)
Test: [450/782]	Time 0.129 (0.136)	Loss 1.1764 (0.9273)	Prec@1 64.062 (77.976)	Prec@5 92.188 (94.049)
Test: [500/782]	Time 0.131 (0.136)	Loss 0.3808 (0.9403)	Prec@1 93.750 (77.656)	Pre

### Full Precision - AR 1.16

In [19]:
val_ar_tfms = [transforms.Resize(int(target_size*1.16)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=64, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_rs_prec5 = validate(val_loader, model, criterion)

Test: [50/782]	Time 0.137 (0.187)	Loss 1.1954 (1.0575)	Prec@1 67.188 (75.406)	Prec@5 89.062 (93.594)
Test: [100/782]	Time 0.130 (0.159)	Loss 1.3734 (1.0582)	Prec@1 57.812 (75.516)	Prec@5 89.062 (93.094)
Test: [150/782]	Time 0.127 (0.150)	Loss 0.7931 (1.0701)	Prec@1 78.125 (75.229)	Prec@5 95.312 (92.771)
Test: [200/782]	Time 0.106 (0.142)	Loss 0.5465 (1.0070)	Prec@1 84.375 (76.344)	Prec@5 100.000 (93.367)
Test: [250/782]	Time 0.110 (0.134)	Loss 0.8014 (1.0016)	Prec@1 79.688 (76.194)	Prec@5 95.312 (93.319)
Test: [300/782]	Time 0.122 (0.132)	Loss 0.9408 (0.9709)	Prec@1 76.562 (76.771)	Prec@5 92.188 (93.542)
Test: [350/782]	Time 0.130 (0.131)	Loss 0.8067 (0.9451)	Prec@1 84.375 (77.438)	Prec@5 95.312 (93.737)
Test: [400/782]	Time 0.129 (0.131)	Loss 0.9111 (0.9264)	Prec@1 81.250 (77.777)	Prec@5 95.312 (93.926)
Test: [450/782]	Time 0.130 (0.131)	Loss 1.1683 (0.9252)	Prec@1 62.500 (77.955)	Prec@5 92.188 (94.122)
Test: [500/782]	Time 0.130 (0.131)	Loss 0.4018 (0.9386)	Prec@1 93.750 (77.638)	Pre

### Full Precision - AR 1.12

In [20]:
val_ar_tfms = [transforms.Resize(int(target_size*1.12)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=64, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_rs_prec5 = validate(val_loader, model, criterion)

Test: [50/782]	Time 0.148 (0.181)	Loss 1.2111 (1.0373)	Prec@1 65.625 (76.188)	Prec@5 92.188 (93.875)
Test: [100/782]	Time 0.131 (0.157)	Loss 1.3915 (1.0472)	Prec@1 56.250 (76.125)	Prec@5 87.500 (93.391)
Test: [150/782]	Time 0.128 (0.149)	Loss 0.7584 (1.0594)	Prec@1 81.250 (75.760)	Prec@5 96.875 (93.083)
Test: [200/782]	Time 0.112 (0.141)	Loss 0.5492 (1.0006)	Prec@1 87.500 (76.711)	Prec@5 100.000 (93.672)
Test: [250/782]	Time 0.111 (0.134)	Loss 0.7393 (0.9966)	Prec@1 82.812 (76.513)	Prec@5 95.312 (93.519)
Test: [300/782]	Time 0.122 (0.131)	Loss 0.9863 (0.9679)	Prec@1 75.000 (76.979)	Prec@5 90.625 (93.698)
Test: [350/782]	Time 0.131 (0.131)	Loss 0.8289 (0.9426)	Prec@1 84.375 (77.589)	Prec@5 93.750 (93.879)
Test: [400/782]	Time 0.130 (0.131)	Loss 0.9314 (0.9246)	Prec@1 81.250 (77.957)	Prec@5 95.312 (94.070)
Test: [450/782]	Time 0.131 (0.132)	Loss 1.1557 (0.9245)	Prec@1 64.062 (78.118)	Prec@5 92.188 (94.267)
Test: [500/782]	Time 0.142 (0.132)	Loss 0.3808 (0.9376)	Prec@1 87.500 (77.772)	Pre

In [21]:
val_ar_tfms = [transforms.Resize(int(target_size*1.13)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=64, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_rs_prec5 = validate(val_loader, model, criterion)

Test: [50/782]	Time 0.136 (0.179)	Loss 1.2334 (1.0421)	Prec@1 64.062 (76.031)	Prec@5 90.625 (93.688)
Test: [100/782]	Time 0.129 (0.155)	Loss 1.3918 (1.0478)	Prec@1 59.375 (76.109)	Prec@5 87.500 (93.344)
Test: [150/782]	Time 0.127 (0.147)	Loss 0.7676 (1.0617)	Prec@1 79.688 (75.625)	Prec@5 96.875 (92.958)
Test: [200/782]	Time 0.106 (0.140)	Loss 0.5427 (1.0020)	Prec@1 85.938 (76.523)	Prec@5 100.000 (93.547)
Test: [250/782]	Time 0.107 (0.133)	Loss 0.7428 (0.9980)	Prec@1 81.250 (76.381)	Prec@5 95.312 (93.431)
Test: [300/782]	Time 0.121 (0.131)	Loss 0.9664 (0.9687)	Prec@1 75.000 (76.969)	Prec@5 92.188 (93.630)
Test: [350/782]	Time 0.129 (0.131)	Loss 0.8253 (0.9433)	Prec@1 84.375 (77.607)	Prec@5 93.750 (93.826)
Test: [400/782]	Time 0.132 (0.131)	Loss 0.9270 (0.9252)	Prec@1 81.250 (77.969)	Prec@5 95.312 (94.012)
Test: [450/782]	Time 0.130 (0.131)	Loss 1.1620 (0.9246)	Prec@1 65.625 (78.087)	Prec@5 92.188 (94.194)
Test: [500/782]	Time 0.130 (0.131)	Loss 0.3930 (0.9377)	Prec@1 90.625 (77.725)	Pre

In [22]:
val_ar_tfms = [transforms.Resize(int(target_size*1.15)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=64, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_rs_prec5 = validate(val_loader, model, criterion)

Test: [50/782]	Time 0.140 (0.181)	Loss 1.2216 (1.0428)	Prec@1 67.188 (75.281)	Prec@5 90.625 (93.781)
Test: [100/782]	Time 0.131 (0.157)	Loss 1.3970 (1.0472)	Prec@1 57.812 (75.812)	Prec@5 85.938 (93.219)
Test: [150/782]	Time 0.128 (0.149)	Loss 0.7578 (1.0622)	Prec@1 81.250 (75.438)	Prec@5 95.312 (92.854)
Test: [200/782]	Time 0.108 (0.142)	Loss 0.5641 (1.0016)	Prec@1 84.375 (76.484)	Prec@5 100.000 (93.453)
Test: [250/782]	Time 0.107 (0.135)	Loss 0.7898 (0.9973)	Prec@1 79.688 (76.406)	Prec@5 95.312 (93.406)
Test: [300/782]	Time 0.123 (0.132)	Loss 0.9487 (0.9669)	Prec@1 75.000 (76.953)	Prec@5 92.188 (93.620)
Test: [350/782]	Time 0.132 (0.132)	Loss 0.7973 (0.9417)	Prec@1 85.938 (77.585)	Prec@5 95.312 (93.768)
Test: [400/782]	Time 0.130 (0.132)	Loss 0.9134 (0.9235)	Prec@1 81.250 (77.902)	Prec@5 95.312 (93.977)
Test: [450/782]	Time 0.130 (0.132)	Loss 1.1607 (0.9233)	Prec@1 62.500 (78.059)	Prec@5 92.188 (94.153)
Test: [500/782]	Time 0.131 (0.132)	Loss 0.3902 (0.9366)	Prec@1 93.750 (77.709)	Pre

In [23]:
val_ar_tfms = [transforms.Resize(int(target_size*1.10)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=64, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_rs_prec5 = validate(val_loader, model, criterion)

Test: [50/782]	Time 0.138 (0.182)	Loss 1.2220 (1.0397)	Prec@1 65.625 (75.750)	Prec@5 87.500 (93.531)
Test: [100/782]	Time 0.131 (0.157)	Loss 1.3749 (1.0492)	Prec@1 57.812 (76.141)	Prec@5 87.500 (93.297)
Test: [150/782]	Time 0.127 (0.149)	Loss 0.7334 (1.0605)	Prec@1 84.375 (75.906)	Prec@5 96.875 (93.052)
Test: [200/782]	Time 0.107 (0.142)	Loss 0.5886 (1.0021)	Prec@1 87.500 (76.750)	Prec@5 100.000 (93.586)
Test: [250/782]	Time 0.108 (0.134)	Loss 0.7451 (0.9975)	Prec@1 84.375 (76.594)	Prec@5 95.312 (93.438)
Test: [300/782]	Time 0.121 (0.131)	Loss 0.9844 (0.9688)	Prec@1 75.000 (77.068)	Prec@5 90.625 (93.651)
Test: [350/782]	Time 0.139 (0.131)	Loss 0.8005 (0.9444)	Prec@1 81.250 (77.701)	Prec@5 93.750 (93.812)
Test: [400/782]	Time 0.130 (0.131)	Loss 0.9365 (0.9271)	Prec@1 79.688 (78.039)	Prec@5 95.312 (93.980)
Test: [450/782]	Time 0.131 (0.131)	Loss 1.1455 (0.9272)	Prec@1 65.625 (78.257)	Prec@5 92.188 (94.139)
Test: [500/782]	Time 0.131 (0.131)	Loss 0.3765 (0.9393)	Prec@1 95.312 (77.912)	Pre

## Comparison

In [ ]:
ar_batch_means = [np.array(c).mean() for c in chunks(ar_means, 10)]

In [ ]:
d = {'DawnValidation': orig_prec5, 
     'BatchAspectRatioValidation': ar_rs_prec5, 
     'AR Mean': ar_batch_means[:-1],
     'Difference': np.array(ar_rs_prec5)-np.array(orig_prec5)}
df = pd.DataFrame(data=d);
df.to_csv('ar_tests.csv')
df

### Test TTA with original validation

In [ ]:
min_scale = 0.5
trn_tfms = [
        transforms.RandomResizedCrop(target_size, scale=(min_scale, 1.0)),
        transforms.RandomHorizontalFlip(),
    ] + tensor_tfm
aug_dataset = datasets.ImageFolder(valdir, transforms.Compose(trn_tfms))

val_tfms = [transforms.Resize(int(target_size*1.14)), transforms.CenterCrop(target_size)] + tensor_tfm
val_dataset = datasets.ImageFolder(valdir,  transforms.Compose(val_tfms))

aug_loader = torch.utils.data.DataLoader(
    aug_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)


In [ ]:
tta_prec5 = validate(val_loader, model, criterion, aug_loader=aug_loader, num_augmentations=4)

In [ ]:
d = {
    'AR Mean': ar_batch_means[:-1],
    'DawnValidation': orig_prec5, 
    'TTA': tta_prec5, 
    'ARValidation': ar_rs_prec5, 
#     'Difference': np.array(ar_rs_prec5)-np.array(orig_prec5)
}
df = pd.DataFrame(data=d);
df.to_csv('ar_tests_tta.csv')
df

### TTA with AR

In [ ]:
min_scale = 0.5
trn_tfms = [
        transforms.RandomResizedCrop(target_size, scale=(min_scale, 1.0)),
        transforms.RandomHorizontalFlip(),
    ] + tensor_tfm
aug_dataset = datasets.ImageFolder(valdir, transforms.Compose(trn_tfms))

aug_loader = torch.utils.data.DataLoader(
    aug_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

val_ar_tfms = [transforms.Resize(int(target_size*1.14)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

In [ ]:
tta_ar_rs_prec5 = validate(val_loader, model, criterion, aug_loader=aug_loader, num_augmentations=4)

In [ ]:
df = pd.read_csv('ar_tests_tta.csv', index_col=0); df

In [ ]:
df_2 = pd.DataFrame({'TTA_AR': tta_ar_rs_prec5})

In [ ]:
df.to_csv('ar_tests_tta.csv')

In [ ]:
df.join(df_2)

In [ ]:
dat1 == pd pd..DataFrameDataFrame({({'dat1''dat1'::  [[99,,55]})]})
dat2 == pd pd..DataFrameDataFrame({({'dat2''dat2'::  [[77,,66]})]})
 >> dat1 dat1..joinjoin((dat2dat2))